In [101]:
import matplotlib.pyplot as plt
import pandas
import os
import spatter_util
import importlib
importlib.reload(spatter_util)

<module 'spatter_util' from '/Users/plavin3/spatter/notebooks/spatter_util.py'>

# Spatter Plots

This notebook will guide you through running standard Spatter testsuites and adding the results to the plots found in our MEMSYS'20 paper, which you can read [here](https://dl.acm.org/doi/abs/10.1145/3422575.3422794) (You should also be able to find a pdf available from Patrick Lavin's Google Scholar page). Specifically, we will be adding your data to Figures 3, 5, and 9.

**Important**: Before you run this notebook, you should have already built Spatter. Please view the README in the top level directory to do so. Once you've built Spatter, note the build directory, and return to this notebook. 

**Note:** Although a new combined Gather/Scatter kernel is planned, it is not yet part of this notebook. 

To get started, fill out the information in the table below about your system. By default, this notebook contains 

In [93]:
# Configuration Information
build_dir      = '../build_omp_gnu' # Relative to the directory containing this notebook
stream_copy_bw =  58543.9           # bw in MB/s - OPTIONAL - set to 0 if you don't know
platform_name  = 'M1'               # Used for plot legends
system_type    = 'cpu'              # Change to 'gpu' if you have build a CUDA or OpenCL backend

## STREAM-like 
First, let's look at how we can use Spatter to get a number similar to what John McCalpin's [STREAM](https://www.cs.virginia.edu/stream/) would give us. 

To accomplish this in Spatter, we want to create a pattern that will read in every element of an array with no reuse. This means we want an index buffer containing the numbers $0$ to $N-1$, and a `delta` of $N$. The value of $N$ does not change the data that is read, but may have performance implications for the architecture you're running on. A good value for CPUs is 8, and a good value for GPUs is 256. We want to run this test as a sanity check before running the more interested tests. 

One caveat when comparing to STREAM is that Spatter is designed to only produce Reads in Gather mode, and to only produce writes in Scatter mode. The numbers should still be similar, however. 

We'll write this one out by hand so we can look at some of the options Spatter supports. After this, we'll just run the scripts that run the standard testsuites. This test will require 1GiB of memory.

In [94]:
N      = 8                  # Index buffer length, as described above
index  = f'-pUNIFORM:{N}:1' # This is a shorthand that Spatter supports meaning "Uniform stride, length N, gap 1
                            # You could also write -p0,1,2,3,4,5,6,7
delta  = f'-d{N}'           # This is the delta applied to the base address between gathers/scatters
length = f'-l{2**24}'       # The -l (--length) option specifies how many gathers or scatters to perform
                            # With an index buffer of length 8, and 8 bytes per element, this will be 2**3 * 2**3 * 2**24 = 1GiB of data read

os.system(f'{build_dir}/spatter {index} {delta} {length} | tee stream-like-read-custom{system_type}.txt')


Running Spatter version 0.4
Compiler: SPAT_C_NAME ver. 14.0.0.14000029
Compiler Location: /usr/bin/gcc
Backend: OPENMP
Aggregate Results? YES

Run Configurations
[ {'name':'UNIFORM', 'kernel':'Gather', 'pattern':[0,1,2,3,4,5,6,7], 'delta':8, 'length':16777216, 'agg':10, 'wrap':1, 'threads':8} ]

config  time(s)      bw(MB/s)    
0       0.01929      55660.2     

Min         25%          Med          75%          Max         
55660.2      55660.2      55660.2      55660.2      55660.2     
H.Mean       H.StdErr    
55660.2      0           


0

### Spatter Output

There are 4 sections in the above output:
1. Build information
1. Run configuations

    - In this case there is only one. Spatter supports a JSON input to specify multiple run configuations. This saves time allocating memory as it can be re-used between benchmark runs. 
        
1. Time and effective bandwidth per configuration. 

    - In the run configuration, you'll see that "agg" is set to 10, meaning the configuation is an aggregate across 10 runs. We take the maximum of all aggregated runs. 
    
1. Summary information

    - These numbers, particularly the harmonic mean, serve as a single performance number for the run configurations.

In [95]:
# Read the output file and get the bandwidth. We'll use a utility function for this further in the notebook
read_bw = float(os.popen(f'grep config --context=1 < stream-like-read-custom{system_type}.txt | tail -n 1 | cut -w -f3').read())
if (stream_copy_bw > 0):
    print(f'Spatter STREAM-like Gather bandwidth is {read_bw} MB/s, which is {100*read_bw/stream_copy_bw:.2f}% of STREAM')
else:
    print(f'Spatter STREAM-like Gather bandwidth is {read_bw} MB/s')

Spatter STREAM-like Gather bandwidth is 55660.2 MB/s, which is 95.07% of STREAM


## STREAM-like Suite

Let's run the STREAM-like suite. We won't make a plot from this data, but we'll use this to see how the other suites are run. These tests will require 1GiB to run.

In [96]:
suite = f'../standard-suite/basic-tests/{system_type}-stream.json'
os.system(f'{build_dir}/spatter -pFILE={suite} | tee stream_custom{system_type}.txt')

Reading patterns from ../standard-suite/basic-tests/cpu-stream.json.

Running Spatter version 0.4
Compiler: SPAT_C_NAME ver. 14.0.0.14000029
Compiler Location: /usr/bin/gcc
Backend: OPENMP
Aggregate Results? YES

Run Configurations
[ {'name':'UNIFORM', 'kernel':'Scatter', 'pattern':[0,1,2,3,4,5,6,7], 'delta':8, 'length':16777216, 'agg':10, 'wrap':1, 'threads':8},
  {'name':'UNIFORM', 'kernel':'Gather', 'pattern':[0,1,2,3,4,5,6,7], 'delta':8, 'length':16777216, 'agg':10, 'wrap':1, 'threads':8} ]

config  time(s)      bw(MB/s)    
0       0.02236      48027.1     
1       0.01893      56727.7     

Min         25%          Med          75%          Max         
48027.1      48027.1      56727.7      56727.7      56727.7     
H.Mean       H.StdErr    
52016.1      3054.91     


0

In [109]:
stream_df = spatter_util.file2df(f'stream_custom{system_type}.txt')
print(f'Scatter Bandwidth: {stream_df["bw(MB/s)"][0]} MB/s')
print(f'Gather  Bandwidth: {stream_df["bw(MB/s)"][1]} MB/s')

Scatter Bandwidth: 48027.1 MB/s
Gather  Bandwidth: 56727.7 MB/s


## Uniform Stride Suite

This suite runs gather and scatter patterns at power of 2 strides, from 1 to 128. Because we are access doubles, a stride of 8 means we are accessing 1 element from every 64-byte cache line. We expect to see effective bandwidth roughly level out at this point, although this is not the case on all systems! You can check out the paper for more on this plot!

This test will require 2GB of memory. The plots in the paper used 16 GB but we want this notebook to run quickly on laptops with less memory.

In [116]:
#suite = f'../standard-suite/basic-tests/{system_type}-ustride.json'
suite = f'cpu_ustride_new.json'
os.system(f'{build_dir}/spatter -pFILE={suite} > ustride_custom{system_type}.txt')
ustride_df = spatter_util.file2df(f'ustride_custom{system_type}.txt')

,config,time(s),bw(MB/s),name,kernel,pattern,delta,length,agg,wrap,threads,arch,archtype,experiment,gap,pat_len,color,pretty
0,0,0.042450,47117.60,UNIFORM,Scatter,"[0, 1, 2, 3, 4, 5, 6, 7]",1,31250000,10,1,8,customcpu,CPU,ustride,0,8,red,CustomCPU
1,1,0.067140,14893.10,UNIFORM,Scatter,"[0, 2, 4, 6, 8, 10, 12, 14]",2,15625000,10,1,8,customcpu,CPU,ustride,1,8,red,CustomCPU
2,2,0.066780,7486.82,UNIFORM,Scatter,"[0, 4, 8, 12, 16, 20, 24, 28]",4,7812500,10,1,8,customcpu,CPU,ustride,2,8,red,CustomCPU
3,3,0.066810,3741.90,UNIFORM,Scatter,"[0, 8, 16, 24, 32, 40, 48, 56]",8,3906250,10,1,8,customcpu,CPU,ustride,3,8,red,CustomCPU
4,4,0.066760,1872.35,UNIFORM,Scatter,"[0, 16, 32, 48, 64, 80, 96, 112]",16,1953125,10,1,8,customcpu,CPU,ustride,4,8,red,CustomCPU
5,5,0.035700,1750.70,UNIFORM,Scatter,"[0, 32, 64, 96, 128, 160, 192, 224]",32,976562,10,1,8,customcpu,CPU,ustride,5,8,red,CustomCPU
6,6,0.017300,1806.67,UNIFORM,Scatter,"[0, 64, 128, 192, 256, 320, 384, 448]",64,488281,10,1,8,customcpu,CPU,ustride,6,8,red,CustomCPU
7,7,0.008570,1823.22,UNIFORM,Scatter,"[0, 128, 256, 384, 512, 640, 768, 896]",128,244140,10,1,8,customcpu,CPU,ustride,7,8,red,CustomCPU
8,8,0.034670,57678.40,UNIFORM,Gather,"[0, 1, 2, 3, 4, 5, 6, 7]",1,31250000,10,1,8,customcpu,CPU,ustride,0,8,red,CustomCPU
9,9,0.034190,29248.30,UNIFORM,Gather,"[0, 2, 4, 6, 8, 10, 12, 14]",2,15625000,10,1,8,customcpu,CPU,ustride,1,8,red,CustomCPU
